In [ ]:
!pip install earthengine-api geemap

In [1]:
#import libraries (earthengine and geemap)
import ee


In [2]:
!earthengine authenticate

In [3]:
# Initialize the Earth Engine module.
ee.Initialize()

In [4]:
# Define the administrative boundary of Pará, Brazil (need to filter grids)
Admin = ee.FeatureCollection("FAO/GAUL/2015/level1")  # Load the GAUL dataset
para_boundary = Admin.filter(ee.Filter.And(
    ee.Filter.eq('ADM1_NAME', 'Para'),
    ee.Filter.eq('ADM0_NAME', 'Brazil')
))

Create a grid (10x10 = 100 tiles in the example) as needed to extract points...

In [11]:
# Define the number of tiles in x and y directions
num_tiles_x = 10  # 10 tiles along the x-axis
num_tiles_y = 10  # 10 tiles along the y-axis

# Calculate the dimensions of each tile
lon_start, lon_end = -59.15588775272494, -45.88440337772494
lat_start, lat_end = -10.054019907719326, 2.812773900498235

lon_edge = (lon_end - lon_start) / num_tiles_x
lat_edge = (lat_end - lat_start) / num_tiles_y

# Create a grid of tiles
polys = []
cell_id = 0
for lon in range(num_tiles_x):
    x1 = lon_start + lon * lon_edge
    x2 = x1 + lon_edge
    for lat in range(num_tiles_y):
        cell_id += 1
        y1 = lat_start + lat * lat_edge
        y2 = y1 + lat_edge
        
        polys.append(ee.Feature(ee.Geometry.Rectangle(x1, y1, x2, y2), {'label': cell_id}))

# Create a FeatureCollection from the list of polygons and Filter the grid to include only tiles that intersect the Pará boundary
grid = ee.FeatureCollection(polys).filterBounds(para_boundary)


Set some functions to filter the data and extract points...

In [13]:
# Define the function to apply quality masks
def apply_quality_mask(image):
    return image.updateMask(image.select('quality_flag').eq(1)).updateMask(image.select('degrade_flag').eq(0))

# Define the function to clip images to the tile geometry
def clip_image_to_tile(image, tile_geometry):
    return image.clip(tile_geometry)

# Function to add latitude and longitude bands to an image.
def add_lat_lon_bands(image):
    lat_lon = ee.Image.pixelLonLat()  # Create an image with latitude and longitude bands.
    return image.addBands(lat_lon)

# Define the function to convert images to points and filter out points with no data
def convert_images_to_points(image):
    geometry = image.geometry()
    points = image.sample(region=geometry, scale=30, numPixels=1e9)
    points = points.filter(ee.Filter.notNull(['rh98']))  # Replace 'rh98' with the correct band name if needed
    points = points.filter(ee.Filter.eq('quality_flag', 1)) # filter points with quality_flag = 1
    points = points.filter(ee.Filter.gte('sensitivity', 0.9)) # sensitivity >= 0.9
    points = points.filter(ee.Filter.inList('beam', [5, 6, 8, 11]))#filter only full power beams.
    return points

The following function works on each tile (one by one) and extract all valid GEDI L2A points converting all raster bands into attribute of the point. It also adds latitude and longitude of the points...

In [14]:
def process_tile(tile):
    tile_geometry = ee.Feature(tile).geometry()

    # Get GEDI L2A raster collection
    gediL2Araster = ee.ImageCollection("LARSE/GEDI/GEDI02_A_002_MONTHLY")

    # Apply quality masks
    gedi_l2a_quality_masked = gediL2Araster.map(apply_quality_mask)

    # Clip images to the tile geometry
    gedi_l2a_clipped = gedi_l2a_quality_masked.map(lambda img: clip_image_to_tile(img, tile_geometry))

    # Add latitude and longitude bands.
    gedi_l2a_with_coords = gedi_l2a_clipped.map(add_lat_lon_bands)

    # Convert images to points.
    gedi_points_tile = gedi_l2a_with_coords.map(convert_images_to_points).flatten()

    # Create a valid description and filename prefix
    tile_label = tile.get('label').getInfo()  # Extract the label value
    description = 'GEDI_L2A_tile_' + str(tile_label)
    filename_prefix = 'tile_' + str(tile_label)

    # Export the points of this tile to a CSV file
    task = ee.batch.Export.table.toDrive(
        collection=gedi_points_tile,
        description=description,
        folder='GEDI_L2A_Para_Brazil',  # Specify your folder name here
        fileNamePrefix=filename_prefix,  # Add a prefix for the file name
        fileFormat='CSV'
    )
    task.start()
    print(f'Started export for tile {tile_label}')



Create a list of tiles from the grid created above and run the process_tile function on each tile...

In [10]:
# Get the list of tiles from the grid
tiles_list = grid.getInfo()['features']

# Relabel the tiles from 1 to the total number (N) of tiles (Optional).
for index, tile in enumerate(tiles_list, start=1):
    tile['properties']['label'] = str(index)

# Process each tile
for tile in tiles_list:
    process_tile(ee.Feature(tile['geometry']).set('label', tile['properties']['label']))


Started export for tile 1
Started export for tile 2
Started export for tile 3
Started export for tile 4
Started export for tile 5
Started export for tile 6
Started export for tile 7
Started export for tile 8
Started export for tile 9
Started export for tile 10
Started export for tile 11
Started export for tile 12
Started export for tile 13
Started export for tile 14
Started export for tile 15
Started export for tile 16
Started export for tile 17
Started export for tile 18
Started export for tile 19
Started export for tile 20
Started export for tile 21
Started export for tile 22
Started export for tile 23
Started export for tile 24
Started export for tile 25
Started export for tile 26
Started export for tile 27
Started export for tile 28
Started export for tile 29
Started export for tile 30
Started export for tile 31
Started export for tile 32
Started export for tile 33
Started export for tile 34
Started export for tile 35
Started export for tile 36
Started export for tile 37
Started ex